# Initial

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyproj
import re
from datetime import date, time, datetime

In [3]:
%run ../Load_Data.ipynb
%run ../Calculation_Functions.ipynb

In [4]:
# This both loads the data from the INGV file,
# and it calculates slip from the provided radius and mL

def loadINGVTest(minMagnitude=0, onlyFocal = False): 
    """Returns: (latitude, longitude, depth,
    mL, radius, ST1, DIP1, RK1, ST2, DIP2, RK2, time, SLIP)
    
    Provide maxMagnitude to load only earthquake with higher than that magnitude
    Only focal is boolean to indicate if earthquake with no focal mechanisms are desired
    
    Angles are returned in radians
    slip and depth are in meters
    """

    # We've had some problems with the address of the file. This should fix it.
    try:
        file=open('LAquila_2009_ALLinONE_unpub.out')
    except:
        url = '/work/Course'#= os.getcwd()
        file=open(url+'/LAquila_2009_ALLinONE_unpub.out')
        
    maxLines = sum(1 for lines in file)
    file.seek(0)
    splitlines = []
    for i in range(maxLines):
        if i == 0:
            next(file)
        else:
            splitlines.append(file.readline().split())
            
    dataSize=len(splitlines)
    year=np.zeros(dataSize,np.int)
    month=np.zeros(dataSize,np.int)    
    day=np.zeros(dataSize,np.int)  
    hours=np.zeros(dataSize,np.int)      
    minutes=np.zeros(dataSize,np.int)    
    seconds=np.zeros(dataSize,np.int)    
    latitude=np.zeros(dataSize,np.float)    
    longitude=np.zeros(dataSize,np.float)  
    depth=np.zeros(dataSize,np.float)      
    ML=np.zeros(dataSize,np.float)    
    err=np.zeros(dataSize,np.float)    
    radius=np.zeros(dataSize,np.float)    
    ID=np.zeros(dataSize,np.int)    
    ST1=np.zeros(dataSize,np.float)   
    DIP1=np.zeros(dataSize,np.float)   
    RK1=np.zeros(dataSize,np.float)   
    ST2=np.zeros(dataSize,np.float)     
    DIP2=np.zeros(dataSize,np.float) 
    RK2=np.zeros(dataSize,np.float)  
    DS=np.zeros(dataSize,np.float)
    tt=np.zeros(dataSize,dtype='datetime64[s]')
    ind = np.zeros(dataSize,np.int)
    fty = np.empty(dataSize, object)
    
    for data in np.arange(dataSize):
        latitude[data]=float(splitlines[data][6])
        longitude[data]=float(splitlines[data][7])    
        depth[data]=float(splitlines[data][8]) * 1000 # meters
        ML[data]=float(splitlines[data][9])
        radius[data]=float(splitlines[data][11]) # meters
        try:
            ind[data]=int(splitlines[data][12])
        except:
            indWithLetter = splitlines[data][12]
            ind[data]=int(re.sub("\D", "", indWithLetter) )         
        ST1[data]=float(splitlines[data][13]) * np.pi / 180 
        DIP1[data]=float(splitlines[data][14]) * np.pi / 180 
        RK1[data]=float(splitlines[data][15]) * np.pi / 180 
        ST2[data]=float(splitlines[data][16]) * np.pi / 180 
        DIP2[data]=float(splitlines[data][17]) * np.pi / 180 
        RK2[data]=float(splitlines[data][18])  * np.pi / 180 
        year[data]=float(splitlines[data][0])
        month[data]=float(splitlines[data][1])
        day[data]=float(splitlines[data][2])
        hours[data]=float(splitlines[data][3])
        minutes[data]=float(splitlines[data][4])
        seconds[data]=float(splitlines[data][5])
        d = date(year[data], month[data], day[data])
        t = time(hours[data], minutes[data])
        dt = datetime.combine(d, t)
        tt[data] = dt
        
        if len(splitlines[data])>19:
            fty[data] = splitlines[data][19]
        else:
            fty[data] = '0'    

    #Here we calculate displacement based on magnitude and radius. 
    #I think we calculated to go from a circular fault to a rectangular fault?
    Mo=10**(1.5*ML+16.1)/1e7
    SLIP = Mo/(2/3*3e10*radius**2*3.14) #slip is displacement in m
    #
    
    #boolean array that selects only the quakes with a rupture mechanism
    if onlyFocal:
        selectQuakesST1 = ~( (ST1 == 0) * (ST2==0) * (DIP1==0) * (DIP2==0) * (RK1==0) * (RK2==0) )
    else:
        selectQuakesST1 = True
        
    selectQuakesML = ML>minMagnitude#max magnitude is the highest magnitude we want to load
    selectQuakes = selectQuakesST1 * selectQuakesML # selectQuakes is boolean for events we want to load
        
    return[ latitude[selectQuakes].copy(), longitude[selectQuakes].copy(), depth[selectQuakes].copy(),
            ML[selectQuakes].copy(), radius[selectQuakes].copy(), ST1[selectQuakes].copy(),
            DIP1[selectQuakes].copy(), RK1[selectQuakes].copy(), ST2[selectQuakes].copy(), DIP2[selectQuakes].copy(),
            RK2[selectQuakes].copy(), tt[selectQuakes].copy(), SLIP[selectQuakes].copy(), 
            fty[selectQuakes.copy()], ind[selectQuakes.copy()] ]

In [5]:
def compInds(ind1, ind2, returnExtra = False):
    mapArr = np.zeros(ind1.shape, dtype = np.int32)
    duplicated = []
    missing = []

    for i in range(ind1.size):
        indMap = np.where(ind1[i]==ind2)[0]
        if indMap.size == 0:
            missing.append([i, ind1[i]])
            mapArr[i] = 0
        elif indMap.size == 1:
            mapArr[i] = indMap[0]
        elif indMap.size > 1:
            mapArr[i] = indMap[0]
            duplicated.append([i, indMap, ind1[i], np.array([ind2[indMap]])])
    if not returnExtra:
        return mapArr
    else:
        return mapArr, duplicated, missing
    

In [6]:
def compInds(ind1, ind2, returnExtra = False):
    mapArr = np.zeros(ind1.shape, dtype = np.int32)
    duplicated = []
    missing = []

    for i in range(ind1.size):
        indMap = np.where(ind1[i]==ind2)[0]
        if indMap.size == 0:
            missing.append([i, ind1[i]])
            mapArr[i] = 0
        elif indMap.size == 1:
            mapArr[i] = indMap[0]
        elif indMap.size > 1:
            mapArr[i] = indMap[0]
            duplicated.append([i, indMap, ind1[i], np.array([ind2[indMap]])])
    if not returnExtra:
        return mapArr
    else:
        return mapArr, duplicated, missing
    

In [7]:
def compInds2(ind1, ind2):
    mapArr12 = []
    mapArr21 = []

    for i in range(ind1.size):
        indMap = np.where(ind1[i]==ind2)[0]
        if indMap.size > 0:
            mapArr12.append(i)   
            mapArr21.append(indMap[0])

    return np.array(mapArr12), np.array(mapArr21)

In [8]:
def mLtomW(mL):
    # From Munafò et al., 2016 - BSSA
    toBig = mL>4
    if toBig.sum() > 0:
        print(toBig.sum(), ' # larger than mL 4. Calculation ran anyway.: mL = ', mL[toBig])
    mW = mL*2/3 + 1.15
        
    return mW

def mWtomO(mW):

    # old implementation commented out
    #     mO = 10 ** (1.5 * mW + 16.1)
    #     mO = mO * 1e-5 * 1e-2 # 1d = 1e-5N   1m = 1e-2cm

    mO = 10 ** (3/2 * (mW + 6.07) ) # Hanks and Kanamori (1979)
    return mO

def mOtomW(mO):
    mW = 2/3 * np.log10(mO) - 6.07
    return mW

def mOtoRad(mO, stressDrop, P):
    """P is Poisson"""
    # Cambiotti described this
    rad = 1/2 * (
            3/2 * 
           (2-P)/(1-P) * 
           (mO / stressDrop)
        
          ) ** (1/3) 
    return rad

def radTomO(rad, stressDrop, P):
    mO = (rad*2) ** 3 * 2/3 * (1-P)/(2-P) * stressDrop
    return mO
    

def slipConstant(shearMod, area, moment):
    # From moment = shearMod * area * slip
    slipConstant = moment / (shearMod * area)
    return slipConstant

def ellipSlipMax(slipAv, area, yMax, xMax):
    # I wrote the solution to this. It's found by suggesting that moment is conserved
    # moving from an constant slip to elliptical slip. 
    sMax = 3/(2 * np.pi) * (slipAv * area) / (yMax * xMax)
    return sMax

# 02-28-2019

In [9]:
[latold, lonold, dold, mold, rold, s1old, 
    d1old, rk1old, s2old, d2old, r2old, told, sold, ftyold, indold
    ] = loadINGVTest()

In [11]:
latold.size

50256

In [12]:
fold     = 'LAquila_2009_ALLinONE_unpub.out'
fnew     = 'LaqAll.final' # 'Brennan_Aquila2009_51k_FocMec_CMT.dat.final'

dnew     = pd.read_csv(fnew, delimiter= '\t', skipinitialspace=True)

mnew     = dnew.mag.values
indnew   = dnew.id_dd.values
latnew   = dnew.lat.values
lonnew   = dnew.lon.values
depthnew = dnew.dep.values
timenew  = dnew.OriginTime.values.astype('datetime64')
ftynew   = dnew.fty.values

In [14]:
mapArrNOList, presentNOList = compInds2(indnew, indold)

In [19]:
stressDrop = 3e6
P = .25
shearMod = 26e9

risbad = np.logical_or((ftynew == 'MP'), (ftynew == '0'))
rCross = np.zeros(lonnew.shape)
mW = np.zeros(lonnew.shape)
rCross[mapArrNOList] = rold[presentNOList]
rCross[risbad] = 0

# have r where it is good. get moment from it. 
mO = radTomO(rCross, stressDrop, P)
# mO = rCross ** 3 * 3e6 * 16 / 7
mO[risbad]=0

# have mO from r... now need mo for other events, get it from ml:
mO[risbad] = mWtomO( mLtomW(mnew[risbad]) )

# have all moments. Now go back to find rad for events where I didn't previously have it. 
rCross[risbad] = mOtoRad(mO[risbad], stressDrop, P)

slip = slipConstant(shearMod, np.pi * rCross**2, mO)

sort = slip.argsort()
print(slip[sort[::-1]])

print('are my units correct?')

1  # larger than mL 4. Calculation ran anyway.: mL =  [4.6]
[0.52385544 0.29435625 0.23516824 ... 0.00097418 0.00090221 0.00090221]
are my units correct?


In [20]:
dMW = dnew.copy()

In [21]:
dMW['radius'] = rCross
dMW['slip'] = slip

In [14]:
dMW.to_csv('LaqAlldf.final', index = False, sep= '\t')

In [15]:
dReload = pd.read_csv('LaqAlldf.final', delimiter= '\t')

# See if things are missing

In [16]:
mapArrNOe, duplicatedNO, indmissingNO = compInds(
    indN, indO, returnExtra = True)
mapArrONe, duplicatedON, indmissingON = compInds(
    indO, indN, returnExtra = True)

NameError: name 'indN' is not defined

In [ ]:
# verify that no HM or JG events are missing
gone = np.array([indmissingON])[0]

missingBoolON = np.zeros(ftyO.shape, dtype = bool)
missingBoolON[gone[:, 0]] = True

print(indO[((ftyO == 'HM') * missingBoolON)]) # if empty, nothing is missing
print(indO[((ftyO == 'JG') * missingBoolON)])

In [ ]:
mNO = mO[mapArrNOe]
mON = mN[mapArrONe]

color = np.empty(mO.size, dtype = object)
color[ftyO == 'HM'] = 'red'
color[ftyO == 'JG'] = 'green'
color[ftyO == 'MP'] = 'blue'
color[ftyO == '0' ] = 'black'

size = 45 * np.ones(mO.size, dtype = float)
size[ftyO == '0'] *= 1/15

plt.scatter(mO, mON, c = color, s = size )
plt.plot([-1, 7],[-1, 7])
plt.xlim((-1, 7))
plt.ylim((-1, 7))

plt.xlabel('m in old file')
plt.ylabel('m in new file where indicies matches those in old file')

plt.show()



plt.figure()

color = np.empty(mN.size, dtype = object)
color[ftyN == 'HM'] = 'red'
color[ftyN == 'JG'] = 'green'
color[ftyN == 'MP'] = 'blue'
color[ftyN == '0' ] = 'black'

size = 45 * np.ones(mN.size, dtype = float)
size[ftyN == '0'] *= 1/15

plt.scatter(mN, mNO, c = color, s = size)
plt.plot([-1, 7],[-1, 7])
plt.xlim((-1, 7))
plt.ylim((-1, 7))

plt.xlabel('m in new file')
plt.ylabel('m in old file where indicies matches those in new file')

plt.show()

In [ ]:
momentNew = radTomO(rCross, 3e6, 0.25)
momentMagNew = mOtomW(momentNew)

# Loading data from online

Description of format: csv: ascii complete description of each single solution. Order of fields is: ev_id, date, time_orig (hh:mm:ss), time_orig_decimals (secs/10), latitude, longitude, depth(km), Mb, Ms, region, source_type, bwave_n_stations, bwave_n_inv, bwave_cutoff, mwave_n_stations, mwave_n_inv, mwave_cutoff, swave_n_stations, swave_n_inv, swave_cutoff, dcouple, centroid_time, delta_centroid_time, centroid_lat, delta_centroid_lat, centroid_long, delta_centroid_long, centroid_depth, delta_centroid_depth, prof, half_dur, inv_date_string, exp, mrr, delta_mrr, mss, delta_mss, mee, delta_mee, mrs, delta_mrs, mre, delta_mre, mse, delta_mse, eigen_t, plunge_t, strike_t, eigen_n, plunge_n, strike_n, eigen_p, plunge_p, strike_p, scalar_moment, strike1, dip1, rake1, strike2, dip2, rake2, Mw, status_flag, quality_flag]

In [ ]:
# load pon file

# Seismic moment tensors of the April 2009, L'Aquila (Central Italy), earthquake sequence
# http://rcmt2.bo.ingv.it/

cmtFile = 'Italydataset1976-2015.csv'
cmtPd = pd.read_csv(cmtFile)
cmtPd.values.T.shape

[ev_id,
date_orig, 
time_orig, 
time_orig_decimals, 
latitude, 
longitude, 
depth, 
Mb, 
Ms, 
region, 
source_type, 
bwave_n_stations, 
bwave_n_inv, 
bwave_cutoff, 
mwave_n_stations, 
mwave_n_inv, 
mwave_cutoff, 
swave_n_stations, 
swave_n_inv, 
swave_cutoff, 
dcouple, 
centroid_time, 
delta_centroid_time, 
centroid_lat, 
delta_centroid_lat, 
centroid_long, 
delta_centroid_long, 
centroid_depth, 
delta_centroid_depth, 
prof, 
half_dur, 
inv_date_string, 
exp, 
mrr, 
delta_mrr,
mss,
delta_mss,
mee,
delta_mee,
mrs,
delta_mrs,
mre,
delta_mre,
mse,
delta_mse,
eigen_t,
plunge_t,
strike_t,
eigen_n,
plunge_n,
strike_n,
eigen_p,
plunge_p,
strike_p,
scalar_moment, 
strike1,
dip1,
rake1,
strike2,
dip2,
rake2,
Mw,
status_flag,
quality_flag] = cmtPd.values.T

In [ ]:
# Filter irrelevant events from the pon file

timepon = (date_orig+'T'+time_orig).astype('datetime64[ms]')
tkeep = ( (timepon>to.min()) * (timepon<to.max()) * 
(latitude<lato.max()) * (latitude>lato.min()) *
 (longitude<lono.max()) * (longitude>lono.min()) )

In [ ]:
# Build function to determin similarity of events.

def normSimilarities(a, b):
    """Finds difference between events as norm of each input quantity."""
    normArr = np.zeros((a.shape[0], b.shape[0]) )
    for i in np.arange(normArr.shape[0]):
        thingsSquared = np.zeros(b.shape[0])
        for j in np.arange(a.shape[1]):
            thingsSquared += (a[i,j]-b[:,j])**2
        normArr[i] = np.sqrt(thingsSquared)
    return normArr

In [ ]:
# making variables to represent events. 


# Get linear spatal coordinate system
latc, lonc = 42, 13
ypon, xpon = ChangeAxis(latc, lonc, latitude, longitude)
dpon = depth * 1000
# ypon = ypon[tkeep]
xpon = xpon[tkeep]
dpon = dpon[tkeep]
yold, xold = ChangeAxis(latc, lonc, lato, lono)
dold = do
ynew, xnew = ChangeAxis(latc, lonc, latN, lonN)
dnew = depthN * 1000


# Magnitude of time of events
mpon = Mw[tkeep].copy()
tpon = timepon[tkeep].astype('datetime64').astype('float64').copy()

mnew = mN.copy() 
tnew = timeN.astype('datetime64').astype('float64').copy()

# Filter events from INGV file which are too small for me to care about

keepINGVNew = mnew>2
xnew = xnew[keepINGVNew]
ynew = ynew[keepINGVNew]
dnew = dnew[keepINGVNew] 
mnew = mnew[keepINGVNew]
tnew = tnew[keepINGVNew]


# scale values to avoid any quantity having to much or little weight

mnew = mnew + 1
print('adding to mnew')
mpon = mpon
mnew *= 1
mpon *= 1

msubtract = mpon.min()
tsubtract = tpon.min()

mpon = mpon - msubtract
mnew = mnew - msubtract
tpon = tpon - tsubtract
tnew = tnew - tsubtract

rangeFactor = xpon.max() - xpon.min()
tFactor = tnew.max() - tnew.min()
mFactor = mnew.max() - mnew.min()

# tpon = tpon / tFactor * rangeFactor
# tnew = tnew / tFactor * rangeFactor
tpon *= 1/1000
tnew *= 1/1000
print('using time as seconds')

mpon = mpon / mFactor * rangeFactor
mnew = mnew / mFactor * rangeFactor

ppon = np.array([xpon, ypon, dpon, tpon, mpon]).T
pnew = np.array([xnew, ynew, dnew, tnew, mnew]).T

ppon = np.array([tpon]).T
pnew = np.array([tnew]).T
print('trying just time')
        
sims = normSimilarities(ppon, pnew)
sims.shape

scaleTime = 1
tpon = tpon * scaleTime
tnew = tnew * scaleTime

In [ ]:
# Make sure that there was a good match
minsim = sims.min()
for i in range(sims.shape[0]):
    sortinds = sims[i].argsort()
    sortinds0 = sortinds[0]
#     print(sortinds[:20]) # which indecies were closest. This should almost look random
    print(sims[i][sortinds][0:5]) # how distinct are the most similar events?
    

In [ ]:
# verify that filtering of pon file seems ok

plt.scatter(xold, yold, c = 'red')
plt.scatter(xpon, ypon,  s = Mw[tkeep].astype('float')**10/100000)